<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1jDaqiq0sISUw0LKfFQ4QkUjgXTuv0ahX?usp=sharing)

# 🛡️ The Guild of Gamecrafters – A Quest Creation Tale
This notebook walks through an interactive, story-driven approach to procedural quest generation using Gemini, CrewAI, FAISS, and more. Each agent in the pipeline is personified with a character and backstory to make the learning journey more engaging.

###🧠 What is RAG and CrewAI?
**RAG or Retrieval-Augmented Generation** is an architecture where an LLM augments its generation by retrieving relevant documents from a vector store like FAISS using embeddings.

**CrewAI** allows you to coordinate multiple specialized AI agents (like Lore Master, Quest Writer, etc.) in a workflow to collaboratively solve complex tasks—like building game quests.

In [ ]:
#Lets start by installing the required Libraries
!pip install -q firecrawl sentence-transformers faiss-cpu google-generativeai crewai nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.8/174.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 6.5 MB/s eta

In [ ]:
#importing the important libraries
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from crewai import Agent, Task, Crew, LLM
from firecrawl import AsyncFirecrawlApp
from IPython.display import Markdown, display
import asyncio
import nest_asyncio


# Replace with your real API keys
genai.configure(api_key="YOUR_GEMINI_API_KEY")
firecrawl_agent = AsyncFirecrawlApp(api_key="YOUR_FIRECRAWL_API_KEY")
gemini = genai.GenerativeModel("gemini-1.5-flash")

my_llm = LLM(
    model='gemini/gemini-1.5-flash',  # ✅ Model string must include provider
    api_key="YOUR_GEMINI_API_KEY"
)

## 📚 The Quest Begins! Pack your stuff for the Adventure Untold.

### Retrieving data from a Wikepedia Page

In [ ]:
nest_asyncio.apply()  # For running async code inside notebooks

async def fetch_firecrawl_data(firecrawl_agent):

    # Await the response properly
    response = await firecrawl_agent.scrape_url(
        url="https://en.wikipedia.org/wiki/Quest_%28video_games%29", #The wikipedia page on game quests...
        formats=["markdown"],
        only_main_content=True,
        parse_pdf=True
    )

    # Safely inspect response
    print("Response attributes:", response.__dict__)

    # Try to extract the markdown content
    try:
        markdown_text = response.markdown  # For newer versions
    except AttributeError:
        print("No .markdown attribute found.")
        return

    chunks = [c.strip() for c in markdown_text.split("\n") if len(c.strip()) > 50]
    return chunks

# Run the async function
wiki_chunks = await fetch_firecrawl_data(firecrawl_agent)



Response attributes: {'url': None, 'markdown': '[Jump to content](https://en.wikipedia.org/wiki/Quest_%28video_games%29#bodyContent)\n\nFrom Wikipedia, the free encyclopedia\n\nPlayer task in video games\n\n"Side Quest" redirects here. For the television series, see [Side Quest (TV series)](https://en.wikipedia.org/wiki/Side_Quest_(TV_series) "Side Quest (TV series)").\n\n|     |     |\n| --- | --- |\n| [![](https://upload.wikimedia.org/wikipedia/en/thumb/9/99/Question_book-new.svg/60px-Question_book-new.svg.png)](https://en.wikipedia.org/wiki/File:Question_book-new.svg) | This article **needs additional citations for [verification](https://en.wikipedia.org/wiki/Wikipedia:Verifiability "Wikipedia:Verifiability")**. Please help [improve this article](https://en.wikipedia.org/wiki/Special:EditPage/Quest_(video_games) "Special:EditPage/Quest (video games)") by [adding citations to reliable sources](https://en.wikipedia.org/wiki/Help:Referencing_for_beginners "Help:Referencing for beginner

### Embedding the Chunks Retrieved

In [ ]:
#The embedding using Sentence TRansformer + FAISS
embedder = SentenceTransformer('all-MiniLM-L6-v2')
lore_chunks = wiki_chunks[:25]
embeddings = embedder.encode(lore_chunks)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

def retrieve_lore(query: str, k: int = 3) -> str:
    query_vec = embedder.encode([query])
    D, I = index.search(query_vec, k)
    return "\n".join([lore_chunks[i] for i in I[0]])

## 🎭The Four Founding Elders

### 📚 Sage Arin, The Lorekeeper

The oldest member of the guild, Sage Arin guards a vast archive of forgotten tales. When presented with a new quest idea, he searches the scrolls for matching legends.

> “Speak your intent,” Arin says, “and I shall search the sands of time.”

In [ ]:
#This agent retrives the lore and expands it into a clear description.
def lore_task_fn(user_prompt):
    prompt = f"""
The following lore was retrieved based on the prompt: "{user_prompt}"

Expand this into a detailed lore description including:
- History, mythology, geography, magic, factions

Return it as a Markdown section titled '## Expanded Lore'.
"""
    return gemini.generate_content(prompt).text


### ✍️ Lady Lyra, The Bard of the Inkflame

Lyra reads the dusty scrolls retrieved by Arin and begins to dream. Her fingers dance across parchment as she turns legends into living stories...

> “A quest,” she hums, “must sing to the heart and scream to the soul.”

In [ ]:
#This agent uses the past retrieved lore and writes the quest.
def quest_task_fn(expanded_lore, user_prompt):
    prompt = f"""
You are a quest designer. Based on the user prompt and expanded lore, create a full quest including characters, mechanics, and progression.

### User Prompt:
{user_prompt}

### Expanded Lore:
{expanded_lore}

Respond in Markdown with sections:
## Quest Overview
## Characters
## Game Mechanics
## Progression (Acts I–III)
## Rewards
"""
    return gemini.generate_content(prompt).text


### ⚖️ Grimbweld Sigen, The Tactician

Grim narrows his eyes. “Too easy and they will yawn. Too hard and they will rage. Let me temper the fire.”

He balances mechanics and rewards with ruthless precision.


In [ ]:
#This agent will ensure that the final reponse id balanced acording to the user's wishes.
def balance_task_fn(quest_markdown):
    prompt = f"""
Balance the following quest for gameplay challenge, pacing, and reward fairness.
Fix issues and return updated Markdown.

{quest_markdown}
"""
    return gemini.generate_content(prompt).text

### ✅ Sir Agustus, The Scribe

The final guardian of the Guild’s name, the Scribe ensures all stories align with canon and shine with clarity.

> “For lore is sacred. Let no tale stray.”

In [ ]:
#This agent polishes the quest generated and ensures crisp clarity.
def qa_task_fn(quest_markdown):
    prompt = f"""
Final QA and polish. Ensure consistency, formatting, and clarity.
Return the final quest scroll in Markdown.

{quest_markdown}
"""
    return gemini.generate_content(prompt).text

## 🧪 The Scroll is Forged

Let us now summon the Guild. Give a prompt, and the guild shall respond with a full quest.


In [ ]:
#This is the function which integrates the agents and generates the Script for the Game Quest.
def run_game_quest_pipeline(user_prompt: str):
    lore_agent = Agent(
        role="Lore Historian",
        goal="Retrieve and expand ancient game lore",
        backstory="Sage Arin, The Lorekeeper is the last of the Lorekeepers, sworn to preserve forgotten tales.",
        llm=my_llm
    )
    lore_task = Task(
        description="Retrieve and expand lore for: " + user_prompt,
        agent=lore_agent,
        expected_output="## Expanded Lore",
        function=lambda: lore_task_fn(user_prompt)
    )

    quest_agent = Agent(
        role="Quest Designer",
        goal="Craft immersive RPG quests",
        backstory="Lady Lyra, The Bard of the Inkflame is a playwright turned adventurer, weaving epic narratives for heroes.",
        llm = my_llm
    )
    quest_task = Task(
        description="Design quest using expanded lore",
        agent=quest_agent,
        expected_output="## Quest Overview",
        function=lambda: quest_task_fn(lore_task.output, user_prompt)
    )

    balance_agent = Agent(
        role="Game Balancer",
        goal="Ensure challenge and reward are balanced",
        backstory="Grimbweld Sigen, The Tactician once judged the Grand Arena. Now he balances power and pace.",
        llm = my_llm
    )
    balance_task = Task(
        description="Balance the quest",
        agent=balance_agent,
        expected_output="Balanced Markdown Quest",
        function=lambda: balance_task_fn(quest_task.output)
    )

    qa_agent = Agent(
        role="Quest Archivist",
        goal="Polish the final quest for clarity and consistency",
        backstory="Sir Agustus, The Scribe served as a royal scribe and ensures no tale is poorly told.",
        llm = my_llm
    )
    qa_task = Task(
        description="Polish and finalize the quest",
        agent=qa_agent,
        expected_output="Final Markdown Quest",
        function=lambda: qa_task_fn(balance_task.output)
    )

    #Here the agents are assesmbled in order, using CrewAI
    crew = Crew(
        agents=[lore_agent, quest_agent, balance_agent, qa_agent],
        tasks=[lore_task, quest_task, balance_task, qa_task]
    )

    results = crew.kickoff()
    return results

## The Final Scroll

In [ ]:
user_prompt = "Design a quest about recovering a legendary sword lost in time" #You can replace this for generating your own Quest...
final_scroll = run_game_quest_pipeline(user_prompt)
display(Markdown(final_scroll.raw))

ERROR:opentelemetry.sdk._shared_internal:Exception while exporting Span.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 493, in _make_request
    conn.request(
  File "/usr/loca

## The Quest for Dawnbreaker: A Complete Guide

**Synopsis:** A fragmented prophecy foretells the return of the Nightshroud, a powerful shadow beast, demanding the reforging of the legendary sword, Dawnbreaker, scattered across time.  As a Heart of Light, the player must journey through temporal rifts, gathering fragments, overcoming challenges, and facing spectral guardians, ultimately reforging the sword and confronting the Nightshroud.  Success hinges on maintaining purity of heart and mastering Dawnbreaker's power.  A morality system subtly impacts difficulty and rewards, with virtuous choices easing challenges and dark deeds creating obstacles.


**I. The Shattered Prophecy (Stage 1)**

* **Objective:** Discover the fragmented prophecy and the pommel fragment of Dawnbreaker.
* **Location:** Ruins of the Order of the Radiant Dawn.
* **Tasks:**
    1. Explore the ruins, deciphering fragmented texts to understand the prophecy and the nature of Dawnbreaker.
    2. Locate a hidden chamber containing the pommel fragment. The fragment will hum, revealing the location of the first temporal rift.
* **Reward:** Pommel fragment (passive increase in light magic, minor shadow magic resistance), a map indicating the first temporal rift's location, and foundational knowledge of Sirus's temporal displacement technology.
* **Difficulty:** Easy


**II. Temporal Rifts:**  Each rift presents unique environmental challenges, puzzles, and spectral guardians.  Successfully navigating a rift awards a Dawnbreaker fragment and lore expanding the world's history.


**A. The Age of Ash (Stage 2): Past**

* **Objective:** Retrieve the blade fragment.
* **Location:** A volcanic wasteland ravaged by past eruptions.
* **Tasks:**
    1. Navigate treacherous terrain, solving environmental puzzles utilizing the volcanic landscape (e.g., redirecting lava flows, utilizing thermal currents).
    2. Overcome fire-based challenges (e.g., avoiding lava, extinguishing fires).
    3. Defeat a powerful fire elemental guarding the hidden temple containing the blade fragment.  Strategic use of light magic is crucial.
* **Reward:** Blade fragment (imbues Dawnbreaker with fire resistance and increased offensive capabilities against fire-based enemies), additional lore regarding Sirus’s struggles during the Sundering.
* **Difficulty:** Moderate


**B. The Obsidian Mire (Stage 3): Past**

* **Objective:** Retrieve the crossguard fragment.
* **Location:** A swamp shrouded in shadow and haunted by wraiths.
* **Tasks:**
    1. Navigate a treacherous swamp, utilizing stealth and light-based abilities to overcome the oppressive darkness.
    2. Solve environmental puzzles involving the manipulation of shadows and light (e.g., using reflections, manipulating light sources to reveal pathways).
    3. Overcome traps and defeat wraiths guarding the crumbling crypt containing the crossguard. Purifying corrupted areas with light magic is essential.
* **Reward:** Crossguard fragment (imbues Dawnbreaker with poison resistance and increased defensive capabilities against shadow-based enemies), additional lore regarding the plague and its impact on the world.
* **Difficulty:** Moderate


**C. The Crystal Caverns (Stage 4): Future**

* **Objective:** Retrieve the hilt fragment.
* **Location:** A technologically advanced cave system where magic and technology intertwine.
* **Tasks:**
    1. Solve technological puzzles involving energy manipulation, decryption, and interaction with advanced constructs.
    2. Navigate complex cave systems utilizing both magical and technological skills.
    3. Overcome a technologically advanced guardian construct protecting the hilt fragment. A combination of magic and technological solutions is required.
* **Reward:** Hilt fragment (allows for technological interface with Dawnbreaker; unlocks additional abilities and enhancements to the sword), further insights into future technologies.
* **Difficulty:** Hard


**D. The Timeless Citadel (Stage 5): Outside Time**

* **Objective:** Retrieve the final blade fragment.
* **Location:** A fortress existing outside the normal flow of time.
* **Tasks:**
    1. Navigate a fortress with shifting corridors and time paradoxes, overcoming temporal anomalies (e.g., time loops, shifting pathways).
    2. Defeat a powerful spectral echo of Sirus – a trial of strength, agility, and skill reflecting Sirus's greatest trials and moral dilemmas.
* **Reward:** Final blade fragment (completes Dawnbreaker, unlocking its full potential), insights into Sirus's character and motivations.
* **Difficulty:** Very Hard


**III. The Reforging (Stage 6)**

* **Objective:** Reforge Dawnbreaker at the Temple of the Radiant Dawn.
* **Location:** A hidden temple within a mystical grove.
* **Tasks:**
    1. Locate the hidden temple, requiring exploration and possibly solving environmental puzzles.
    2. Gather specific resources necessary for the reforging ritual.
    3. Complete the reforging ritual by channeling the player’s own light energy, reflecting their purity of heart and accumulated light magic.
* **Reward:** Reforged Dawnbreaker (possesses full power, allowing players access to various enhanced light-based spells and abilities), enhanced stats and abilities based on player choices and actions throughout the quest.  The sword's appearance may subtly reflect the player's morality.
* **Difficulty:** Moderate


**IV. Confronting the Nightshroud (Stage 7)**

* **Objective:** Confront and defeat the Nightshroud using the reforged Dawnbreaker.
* **Location:** The dimensional rift where the Nightshroud is imprisoned.
* **Tasks:** Engage in a challenging boss battle against the Nightshroud, utilizing the full capabilities of the reforged Dawnbreaker. The battle requires strategic use of abilities, adaptability to the Nightshroud's attacks, and an understanding of its weaknesses.  The difficulty scales subtly based on player choices throughout the quest.
* **Reward:** Victory over the Nightshroud, restoration of balance to Aerthos, recognition as a hero, and an epilogue detailing the player's contribution to history. The epilogue will vary based on choices and actions throughout the quest.
* **Difficulty:** Very Hard


**Failure Conditions:**

* Failure to retrieve all fragments within an implied timeframe (game-defined).
* Corruption of the player's heart through succumbing to darkness or making morally questionable choices (impacting difficulty and potentially triggering negative consequences).
* Defeat at the hands of a guardian or the Nightshroud.


**Success Conditions:**

* Successful retrieval of all fragments and completion of the reforging ritual.
* Defeat of the Nightshroud.
* Maintaining a relatively pure heart throughout the quest (resulting in a more positive epilogue).


This detailed quest provides a comprehensive and engaging experience, blending action, exploration, puzzle-solving, and moral dilemmas within a rich lore.  The player’s choices directly impact the outcome, creating a truly immersive and memorable RPG experience.

###Now tell me o Adventurer, what will be Your Quest?...